# Blocking: Attribute Equivalence Blocking

In [1]:
import sys
sys.path.insert(1, '../../')

from entityresolution.datasets import fetch_DBLP_ACM
from entityresolution.indexing.block.building import AttributeEquivalenceBlocking, SortedNeighborhoodBlocking


## load dataset

In [2]:
golden_standard, amc, dblp  = fetch_DBLP_ACM()

  0%|          | 0.00/270k [00:00<?, ?B/s]

In [3]:
amc.head(3)

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999


In [4]:
dblp.head(3)

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002


In [5]:
amc = amc.set_index("id")
dblp = dblp.set_index("id")

ae = AttributeEquivalenceBlocking(left_on="title", right_on="title")
ae.build(amc, dblp).head()

title_l  \
rec_l  rec_r                                                                 
277955 conf/sigmod/Epstein95                               Keynote Address   
       conf/sigmod/Ellison95                               Keynote Address   
276377 conf/sigmod/OlstonWACELSS98                              DataSplash   
276367 conf/sigmod/Aulakh98               About Quark Digital Media System   
304211 journals/vldb/MaratheS02     Query processing techniques for arrays   

                                                                            authors_l  \
rec_l  rec_r                                                                            
277955 conf/sigmod/Epstein95                                        Robert S. Epstein   
       conf/sigmod/Ellison95                                        Robert S. Epstein   
276377 conf/sigmod/OlstonWACELSS98  Chris Olston, Allison Woodruff, Alexander Aike...   
276367 conf/sigmod/Aulakh98                                              Kamar Aulakh   
304211 journals/vldb/MaratheS02                  Arunprasad P. Marathe, Kenneth Salem   

                                                                           venue_l  \
rec_l  rec_r                                                                         
277955 conf/sigmod/Epstein95        International Conference on Management of Data   
       conf/sigmod/Ellison95        International Conference on Management of Data   
276377 conf/sigmod/OlstonWACELSS98  International Conference on Management of Data   
276367 conf/sigmod/Aulakh98         International Conference on Management of Data   
304211 journals/vldb/MaratheS02     International Conference on Management of Data   

                                    year_l  \
rec_l  rec_r                                 
277955 conf/sigmod/Epstein95          1995   
       conf/sigmod/Ellison95          1995   
276377 conf/sigmod/OlstonWACELSS98    1998   
276367 conf/sigmod/Aulakh98           1998   
304211 journals/vldb/MaratheS02       1999   

                                                                   title_r  \
rec_l  rec_r                                                                 
277955 conf/sigmod/Epstein95                               Keynote Address   
       conf/sigmod/Ellison95                               Keynote Address   
276377 conf/sigmod/OlstonWACELSS98                              DataSplash   
276367 conf/sigmod/Aulakh98               About Quark Digital Media System   
304211 journals/vldb/MaratheS02     Query processing techniques for arrays   

                                                                            authors_r  \
rec_l  rec_r                                                                            
277955 conf/sigmod/Epstein95                                        Robert S. Epstein   
       conf/sigmod/Ellison95                                            Larry Ellison   
276377 conf/sigmod/OlstonWACELSS98  Vuk Ercegovac, Allison Woodruff, Michael Stone...   
276367 conf/sigmod/Aulakh98                                              Kamar Aulakh   
304211 journals/vldb/MaratheS02                  Arunprasad P. Marathe, Kenneth Salem   

                                              venue_r  year_r  \
rec_l  rec_r                                                    
277955 conf/sigmod/Epstein95        SIGMOD Conference    1995   
       conf/sigmod/Ellison95        SIGMOD Conference    1995   
276377 conf/sigmod/OlstonWACELSS98  SIGMOD Conference    1998   
276367 conf/sigmod/Aulakh98         SIGMOD Conference    1998   
304211 journals/vldb/MaratheS02               VLDB J.    2002   

                                                                  AE_title  
rec_l  rec_r                                                                
277955 conf/sigmod/Epstein95                               Keynote Address  
       conf/sigmod/Ellison95                               Keynote Address  
276377 conf/sigmod/OlstonWACELSS98 

In [6]:
sb = SortedNeighborhoodBlocking("title")
df_sb = sb.build(amc)

(6876, 2)


In [8]:

df_sb.head(20)

title_l  \
index_l index_r                                                      
671000  672996   1-Safe Algorithms for Symmetric Site Configura...   
        245901   1-Safe Algorithms for Symmetric Site Configura...   
672996  245901   2Q: A Low Overhead High Performance Buffer Man...   
        245901   2Q: A Low Overhead High Performance Buffer Man...   
        276366   2Q: A Low Overhead High Performance Buffer Man...   
245901  276366                      3D geographic network displays   
        276366                      3D geographic network displays   
        671702                      3D geographic network displays   
276366  671702   50,000 users on an Oracle8 universal server da...   
        671702   50,000 users on an Oracle8 universal server da...   
        775455   50,000 users on an Oracle8 universal server da...   
671702  775455                    A 20/20 Vision of the VLDB-2020?   
        775455                    A 20/20 Vision of the VLDB-2020?   
        671693                    A 20/20 Vision of the VLDB-2020?   
775455  671693   A Bayesian decision model for cost optimal rec...   
        671693   A Bayesian decision model for cost optimal rec...   
        672364   A Bayesian decision model for cost optimal rec...   
671693  672364    A Case-Based Approach to Information Integration   
        672364    A Case-Based Approach to Information Integration   
        673169    A Case-Based Approach to Information Integration   

                                                         authors_l  \
index_l index_r                                                      
671000  672996   Rune Humborstad, Maitrayi Sabaratnam, Svein-Ol...   
        245901   Rune Humborstad, Maitrayi Sabaratnam, Svein-Ol...   
672996  245901                     Theodore Johnson, Dennis Shasha   
        245901                     Theodore Johnson, Dennis Shasha   
        276366                     Theodore Johnson, Dennis Shasha   
245901  276366         Kenneth C. Cox, Stephen G. Eick, Taosong He   
        276366         Kenneth C. Cox, Stephen G. Eick, Taosong He   
        671702         Kenneth C. Cox, Stephen G. Eick, Taosong He   
276366  671702   Tirthankar Lahiri, Ashok Joshi, Amit Jasuja, S...   
        671702   Tirthankar Lahiri, Ashok Joshi, Amit Jasuja, S...   
        775455   Tirthankar Lahiri, Ashok Joshi, Amit Jasuja, S...   
671702  775455   S. Misbah Deen, Anant Jhingran, Shamkant B. Na...   
        775455   S. Misbah Deen, Anant Jhingran, Shamkant B. Na...   
        671693   S. Misbah Deen, Anant Jhingran, Shamkant B. Na...   
775455  671693     V. S. Verykios, G. V. Moustakides, M. G. Elfeky   
        671693     V. S. Verykios, G. V. Moustakides, M. G. Elfeky   
        672364     V. S. Verykios, G. V. Moustakides, M. G. Elfeky   
671693  672364      Maurizio Panti, Luca Spalazzi, Alberto Giretti   
        672364      Maurizio Panti, Luca Spalazzi, Alberto Giretti   
        673169      Maurizio Panti, Luca Spalazzi, Alberto Giretti   

                                                           venue_l  year_l  \
index_l index_r                                                              
671000  672996                               Very Large Data Bases    1997   
        245901                               Very Large Data Bases    1997   
672996  245901                               Very Large Data Bases    1994   
        245901                               Very Large Data Bases    1994   
        276366                               Very Large Data Bases    1994   
245901  276366                                  ACM SIGMOD Record     1996   
        276366                                  ACM SIGMOD Record     1996   
        671702                                  ACM SIGMOD Record     1996   
276366  671702      International Conference on Management of Data    1998   
        671702      International Conference on Management of Data    1998   
        775455      International Confere